# README

## Overview

This notebook visualizes **monthly and seasonal Sea Surface Temperature (SST) climatologies** and their **gradients** using SST data from NetCDF files and port locations from CSV files. It produces:

- **12-panel monthly SST plots** with isolines and port markers.  
- **12-panel monthly SST gradient plots** highlighting temperature fronts.  
- **4-panel seasonal averages** showing SST and gradient maps for summer and winter.

## Features

- Loads SST and port data.  
- Calculates SST gradients and seasonal averages.  
- Generates maps with shading, contours, and labeled ports.  
- Saves figures to specified paths.

## Outputs

- `sst_climatology_monthly.png`: Monthly SST climatology (12 subplots).  
- `sst_gradient_monthly.png`: Monthly SST gradient magnitude (12 subplots).  
- `sst_seasonal_averages.png`: Seasonal averages of SST and gradient (4 subplots).  

These plots provide insight into SST patterns, variability, and frontal structures throughout the year.


In [43]:

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import calendar
import cmocean
import pandas as pd

def load_sst_data(file_path):
    """
    Load Sea Surface Temperature (SST) data from a NetCDF file.

    Parameters:
        file_path (str): Path to the NetCDF file containing SST data.

    Returns:
        xarray.Dataset: Dataset containing SST and associated coordinates.
    """
    return xr.open_dataset(file_path)

def load_ports_data(file_path):
    """
    Load port location data from a CSV file.

    Parameters:
        file_path (str): Path to the CSV file containing port information.

    Returns:
        pandas.DataFrame: DataFrame with port names and their geographical coordinates.
    """
    return pd.read_csv(file_path, index_col=False)

def calculate_sst_gradient(sst_array, lat_array, lon_array):
    """
    Calculate the gradient magnitude of Sea Surface Temperature (SST).

    Parameters:
        sst_array (numpy.ndarray): 2D array of SST values.
        lat_array (numpy.ndarray): 1D array of latitude values.
        lon_array (numpy.ndarray): 1D array of longitude values.

    Returns:
        numpy.ndarray: 2D array of SST gradient magnitudes.

    Notes:
        The function uses numpy's gradient to compute spatial gradients along latitude and longitude.
    """
    # dSST_dlat, dSST_dlon = np.gradient(sst_array, lat_array, lon_array, axis=(0, 1))
    # return np.sqrt(dSST_dlat ** 2 + dSST_dlon ** 2)

    dlat_km = 111.32  # 1° latitud ≈ 111.32 km
    dlon_km = 111.32 * np.cos(np.deg2rad(lat_array.mean()))  # Considera la latitud media para simplificar

    # Usar las distancias calculadas como espaciamiento en np.gradient
    dSST_dlat, dSST_dlon = np.gradient(sst_array, dlat_km, dlon_km, axis=(0, 1))

    return np.sqrt(dSST_dlat ** 2 + dSST_dlon ** 2)

def configure_ticks(ax, lon, lat):
    """
    Configure the longitude and latitude ticks for a given axis.

    Parameters:
        ax (matplotlib.axes.Axes): Axes object to configure.
        lon (numpy.ndarray): Array of longitude values.
        lat (numpy.ndarray): Array of latitude values.

    Description:
        - Longitudes are shifted to the [-180, 180] range.
        - Tick intervals are determined based on the range of lat/lon.
        - A maximum of 5 ticks are placed along each axis with a minimum spacing of 0.5 degrees.

    Returns:
        None
    """
    lon = (lon + 180) % 360 - 180  # Adjust longitudes to standard range

    lon_min, lon_max = np.round(lon.min() * 2) / 2, np.round(lon.max() * 2) / 2
    lat_min, lat_max = np.round(lat.min() * 2) / 2, np.round(lat.max() * 2) / 2

    lon_ticks = np.linspace(lon_min, lon_max, num=min(5, int((lon_max - lon_min) / 0.5) + 1))
    lat_ticks = np.linspace(lat_min, lat_max, num=min(5, int((lat_max - lat_min) / 0.5) + 1))

    ax.set_xticks(lon_ticks, crs=ccrs.PlateCarree())
    ax.set_yticks(lat_ticks, crs=ccrs.PlateCarree())

    ax.set_xticklabels([f"{x:.1f}°" for x in lon_ticks], fontsize=6)
    ax.set_yticklabels([f"{y:.1f}°" for y in lat_ticks], fontsize=6)

    ax.tick_params(axis='both', which='both', length=3, width=0.5)

def plot_with_isolines(ax, lon, lat, data, cmap, vmin, vmax, title, contour_levels, df_ports=None, port_names=None, show_isolines=True):
    """
    Plots data with optional isolines, shading, geographic features, and optional port markers.

    Parameters:
        ax (matplotlib.axes.Axes): Plotting axis.
        lon, lat (numpy.ndarray): Longitude and latitude arrays.
        data (numpy.ndarray): 2D data to plot.
        cmap (Colormap): Colormap for shading.
        vmin, vmax (float): Color scale limits.
        title (str): Plot title.
        contour_levels (array-like): Levels for contour lines.
        df_ports (DataFrame, optional): Port data with coordinates.
        port_names (list, optional): Ports to display.
        show_isolines (bool, optional): Whether to display contour isolines. Default is True.

    Returns:
        QuadMesh: pcolormesh object for colorbar reference.
    """

    im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())

    if show_isolines:
        contours = ax.contour(lon, lat, data, levels=contour_levels, colors='k', linewidths=0.4, transform=ccrs.PlateCarree())
        ax.clabel(contours, inline=True, fontsize=6, fmt="%.1f")

    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    configure_ticks(ax, lon, lat)
    ax.set_title(title, fontsize=9)

    if df_ports is not None and port_names is not None:
        ports = df_ports[df_ports['caleta'].isin(port_names)]
        ax.scatter(ports['lon_decimal_corrected'], ports['lat_decimal_corrected'], 
                   color='blue', edgecolor='black', s=40, transform=ccrs.PlateCarree(), zorder=5)
        for _, row in ports.iterrows():
            ax.text(row['lon_decimal_corrected'] + 0.2, row['lat_decimal_corrected'], row['caleta'],
                    fontsize=5, ha='left', va='center', transform=ccrs.PlateCarree(),
                    bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, pad=0.2))

    return im


def create_axes_layout(fig, axes_locs):
    return [fig.add_axes(loc, projection=ccrs.PlateCarree()) for loc in axes_locs]

def plot_monthly_sst(ds_climatology, sst_var, lat_var, lon_var, df_ports, port_names, save_path):
    """
    Plots monthly Sea Surface Temperature (SST) climatology in a 12-panel layout.

    Parameters:
        ds_climatology (xarray.Dataset): Dataset containing SST and coordinate variables.
        sst_var (str): Name of the SST variable in the dataset.
        lat_var (str): Name of the latitude variable.
        lon_var (str): Name of the longitude variable.
        df_ports (DataFrame): Port data with coordinates.
        port_names (list): Ports to display on the plots.
        save_path (str): Path to save the output figure.

    Description:
        - Generates 12 subplots (one per month) with SST shading, isolines, and port locations.
        - Adds a vertical colorbar indicating temperature scale.
        - Saves the figure to the specified path.
    """

    sst = ds_climatology[sst_var]
    lat = ds_climatology[lat_var].values
    lon = ds_climatology[lon_var].values

    axes_locs = [
        [0.05, 0.72, 0.2, 0.25], [0.27, 0.72, 0.2, 0.25], [0.49, 0.72, 0.2, 0.25], [0.71, 0.72, 0.2, 0.25],
        [0.05, 0.41, 0.2, 0.25], [0.27, 0.41, 0.2, 0.25], [0.49, 0.41, 0.2, 0.25], [0.71, 0.41, 0.2, 0.25],
        [0.05, 0.10, 0.2, 0.25], [0.27, 0.10, 0.2, 0.25], [0.49, 0.10, 0.2, 0.25], [0.71, 0.10, 0.2, 0.25]
    ]

    vmin, vmax = np.round(sst.min().item(), 0), np.round(sst.max().item(), 0)
    vmin, vmax = 15, 27

    contour_levels = np.arange(vmin, vmax + 0.5, 0.5)

    fig = plt.figure(figsize=(16, 12))
    axes = create_axes_layout(fig, axes_locs)
    cmap = cmocean.cm.thermal

    for i, ax in enumerate(axes):
        plot_with_isolines(ax, lon, lat, sst.isel(time=i).values, cmap, vmin, vmax, calendar.month_abbr[i + 1], contour_levels, df_ports, port_names, show_isolines=True)

    cbar_ax = fig.add_axes([0.94, 0.1, 0.015, 0.8])
    cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax)),
                        cax=cbar_ax, orientation='vertical', extend='both')
    cbar.set_label("Sea Surface Temperature (°C)", fontsize=9)
    cbar.ax.tick_params(labelsize=7)

    fig.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

def plot_monthly_gradient(ds_climatology, sst_var, lat_var, lon_var, threshold_percentile, df_ports, port_names, save_path):

    """
    Plots the monthly gradient magnitude of Sea Surface Temperature (SST) in a 12-panel layout.

    Parameters:
        ds_climatology (xarray.Dataset): Dataset containing SST and coordinate variables.
        sst_var (str): Name of the SST variable in the dataset.
        lat_var (str): Name of the latitude variable.
        lon_var (str): Name of the longitude variable.
        threshold_percentile (float): Percentile threshold to mask lower gradient values.
        df_ports (DataFrame): Port data with coordinates.
        port_names (list): Ports to display on the plots.
        save_path (str): Path to save the output figure.

    Description:
        - Calculates monthly SST gradients and masks values below the specified percentile.
        - Generates 12 subplots (one per month) with gradient shading, isolines, and port locations.
        - Adds a vertical colorbar indicating gradient magnitude.
        - Saves the figure to the specified path.
    """

    sst = ds_climatology[sst_var]
    lat = ds_climatology[lat_var].values
    lon = ds_climatology[lon_var].values

    gradients = [calculate_sst_gradient(sst.isel(time=i).values, lat, lon) for i in range(12)]
    gradient_ds = np.array(gradients)

    threshold = np.nanpercentile(gradient_ds, threshold_percentile)
    masked_gradient = np.where(gradient_ds > threshold, gradient_ds, np.nan)

    vmin, vmax = 0, np.ceil(np.nanmax(masked_gradient))
    vmin, vmax = 0, 0.01

    contour_levels = np.linspace(vmin, vmax, 10)

    axes_locs = [
        [0.05, 0.72, 0.2, 0.25], [0.27, 0.72, 0.2, 0.25], [0.49, 0.72, 0.2, 0.25], [0.71, 0.72, 0.2, 0.25],
        [0.05, 0.41, 0.2, 0.25], [0.27, 0.41, 0.2, 0.25], [0.49, 0.41, 0.2, 0.25], [0.71, 0.41, 0.2, 0.25],
        [0.05, 0.10, 0.2, 0.25], [0.27, 0.10, 0.2, 0.25], [0.49, 0.10, 0.2, 0.25], [0.71, 0.10, 0.2, 0.25]
    ]

    fig = plt.figure(figsize=(16, 12))
    axes = create_axes_layout(fig, axes_locs)
    cmap = cmocean.cm.amp

    for i, ax in enumerate(axes):
        plot_with_isolines(ax, lon, lat, masked_gradient[i], cmap, vmin, vmax, f"Gradient - {calendar.month_abbr[i + 1]}", contour_levels, df_ports, port_names, show_isolines=False)

    cbar_ax = fig.add_axes([0.94, 0.1, 0.015, 0.8])
    cbar = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax)),
                        cax=cbar_ax, orientation='vertical', extend='both')
    cbar.set_label("SST Gradient Magnitude (°C/km)", fontsize=9)
    cbar.ax.tick_params(labelsize=7)

    fig.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

def plot_seasonal_averages(ds_climatology, sst_var, lat_var, lon_var, df_ports, port_names, save_path):
    """
    Plots seasonal averages of Sea Surface Temperature (SST) and its gradient in a 4-panel layout.

    Parameters:
        ds_climatology (xarray.Dataset): Dataset containing SST and coordinate variables.
        sst_var (str): Name of the SST variable in the dataset.
        lat_var (str): Name of the latitude variable.
        lon_var (str): Name of the longitude variable.
        df_ports (DataFrame): Port data with coordinates.
        port_names (list): Ports to display on the plots.
        save_path (str): Path to save the output figure.

    Description:
        - Calculates seasonal averages of SST and its gradient.
        - Generates 4 subplots: two seasons with SST and gradient maps.
        - Adds isolines, port locations, and a shared colorbar.
        - Saves the figure to the specified path.
    """
    sst = ds_climatology[sst_var]
    lat = ds_climatology[lat_var].values
    lon = ds_climatology[lon_var].values

    seasons = {
        "Summer (Oct-Ene)": [10, 11, 12, 1],
        "Winter (Abr-Jul)": [4, 5, 6, 7]
    }

    axes_locs = [
        [0.05, 0.55, 0.4, 0.4],  # Summer SST
        [0.05, 0.05, 0.4, 0.4],  # Winter SST
        [0.55, 0.55, 0.4, 0.4],  # Summer Gradient
        [0.55, 0.05, 0.4, 0.4]   # Winter Gradient
    ]

    fig = plt.figure(figsize=(10, 10))
    cmap_sst = cmocean.cm.thermal
    cmap_grad = cmocean.cm.amp

    for i, (season_name, months) in enumerate(seasons.items()):
        sst_season = sst.sel(time=sst['time'].dt.month.isin(months)).mean(dim='time')
        vmin_sst, vmax_sst = np.round(sst_season.min().item(), 0), np.round(sst_season.max().item(), 0)
        vmin_sst, vmax_sst = 15, 27
        contour_levels_sst = np.arange(vmin_sst, vmax_sst + 0.5, 0.5)

        ax_sst = fig.add_axes(axes_locs[i * 2], projection=ccrs.PlateCarree())
        plot_with_isolines(ax_sst, lon, lat, sst_season.values, cmap_sst, vmin_sst, vmax_sst,
                           f"SST - {season_name}", contour_levels_sst, df_ports, port_names, show_isolines=True)

        gradient_season = calculate_sst_gradient(sst_season.values, lat, lon)
        vmin_grad, vmax_grad = 0, np.ceil(np.nanmax(gradient_season))
        vmin_grad, vmax_grad = 0, 0.01

        contour_levels_grad = np.linspace(vmin_grad, vmax_grad, 10)

        ax_grad = fig.add_axes(axes_locs[i * 2 + 1], projection=ccrs.PlateCarree())
        plot_with_isolines(ax_grad, lon, lat, gradient_season, cmap_grad, vmin_grad, vmax_grad,
                           f"Gradient - {season_name}", contour_levels_grad, df_ports, port_names, show_isolines=True)

    cbar_ax_sst = fig.add_axes([0.97, 0.55, 0.015, 0.4])
    cbar_sst = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap_sst, norm=plt.Normalize(vmin=vmin_sst, vmax=vmax_sst)),
                            cax=cbar_ax_sst, orientation='vertical', extend='both')
    cbar_sst.set_label("SST (°C)", fontsize=9)
    cbar_sst.ax.tick_params(labelsize=7)

    cbar_ax_grad = fig.add_axes([0.97, 0.05, 0.015, 0.4])
    cbar_grad = fig.colorbar(plt.cm.ScalarMappable(cmap=cmap_grad, norm=plt.Normalize(vmin=vmin_grad, vmax=vmax_grad)),
                             cax=cbar_ax_grad, orientation='vertical', extend='both')
    cbar_grad.set_label("SST Gradient (°C/km)", fontsize=9)
    cbar_grad.ax.tick_params(labelsize=7)

    fig.savefig(save_path, dpi=400, bbox_inches='tight')
    plt.close(fig)


In [44]:
def main(file_path_sst, file_path_ports, output_dir, port_names, threshold_percentile=90):
    """
    Processes SST data to generate monthly climatology, gradient plots, and seasonal averages.

    Parameters:
        file_path_sst (str): Path to the SST NetCDF file.
        file_path_ports (str): Path to the ports CSV file.
        output_dir (str): Directory to save output figures.
        port_names (list): Ports to display on the plots.
        threshold_percentile (float): Percentile threshold for gradient masking.
    """
    import os

    os.makedirs(output_dir, exist_ok=True)

    ds_climatology = load_sst_data(file_path_sst)
    df_ports = load_ports_data(file_path_ports)

    sst_var, lat_var, lon_var = 'sst', 'lat', 'lon'

    plot_monthly_sst(ds_climatology, sst_var, lat_var, lon_var, df_ports, port_names, 
                     f"{output_dir}/sst_climatology_monthly.png")

    plot_monthly_gradient(ds_climatology, sst_var, lat_var, lon_var, threshold_percentile, df_ports, port_names, 
                          f"{output_dir}/sst_gradient_monthly.png")

    plot_seasonal_averages(ds_climatology, sst_var, lat_var, lon_var, df_ports, port_names, 
                           f"{output_dir}/sst_seasonal_averages.png")


if __name__ == "__main__":
    # Parámetros comunes
    file_path_ports = '../data/puertos/processed/ports_with_normal_angles_corrected.csv'
    port_names = ['Caleta  tortuga (Paita)', 'Caleta Parachique', 
                  'Puerto Chimbote', 'Puerto Samanco', 
                  'Puerto Huarmey', 'Puerto Supe', 'DPA Huacho - FONDEPES', 'Puerto Chancay', 'DPA Callao', 
                  'Puerto Tambo de Mora', 'Terminal Marino Pisco-Camisea (Pluspetro)', 
                  'Caleta Atico', 'Caleta Planchada', 
                  'DPA Ilo']

    # Base de datos OISSTv2
    main(
        file_path_sst='../data/ocean_data_sst/processed/sst_climatology_monthly_1990_2020.nc',
        file_path_ports=file_path_ports,
        output_dir='../docs/figs/oisstv2',
        port_names=port_names
    )

    # Base de datos MODIS
    main(
        file_path_sst='../data/MODIS/processed/sst_climatology_monthly_2003_2023.nc',
        file_path_ports=file_path_ports,
        output_dir='../docs/figs/MODIS',
        port_names=port_names
    )
